In [78]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font

# Function to find the relevant columns and the "ANNUAL TOTAL OR MEAN" row
def find_relevant_data(sheet):
    fog_col, thunder_col, dust_storm_col = None, None, None
    total_row_index = None

    # Find the "ANNUAL TOTAL OR MEAN" row
    for idx, row in sheet.iterrows():
        if row.astype(str).str.contains('ANNUAL TOTAL OR MEAN', case=False, na=False).any():
            total_row_index = idx
            break

    # Find the columns for FOG, THUNDER, and DUST STORM
    for col in sheet.columns:
        if 'FOG' in sheet[col].astype(str).values:
            fog_col = col
        if 'THUNDER' in sheet[col].astype(str).values:
            thunder_col = col
        if 'DUST STORM' in sheet[col].astype(str).values:
            dust_storm_col = col

    return fog_col, thunder_col, dust_storm_col, total_row_index

# Load the Excel file
file_path = 'B:\\Geo Project\\Ramanan Sir\\Tamilnadu_Climate\\Tamilnadu\\Rainfall_book-vellore.xlsx'  # Replace with your file path

# Load the sheet with the correct header row
sheet = pd.read_excel(file_path, header=4)

# Extract the station name from cell A2
wb = load_workbook(file_path, data_only=True)
ws = wb.active
station_name = ws['A2'].value

# Find the relevant columns and the row with "ANNUAL TOTAL OR MEAN"
fog_col, thunder_col, dust_storm_col, total_row_index = find_relevant_data(sheet)

# Prepare the results
if fog_col and thunder_col and dust_storm_col and total_row_index is not None:
    fog_value = sheet.at[total_row_index, fog_col]
    thunder_value = sheet.at[total_row_index, thunder_col]
    dust_storm_value = sheet.at[total_row_index, dust_storm_col]

    # Create a DataFrame with correct column names
    results_df = pd.DataFrame({
        'STATION': [station_name],
        'THUNDER': [thunder_value],
        'FOG': [fog_value],
        'DUST': [dust_storm_value]
    })

    # Save the DataFrame to a new Excel file with formatting
    output_file_path = 'B:\\Geo Project\\Ramanan Sir\\T final\\vellore climate.xlsx'  # Replace with your output file path
    results_df.to_excel(output_file_path, engine='openpyxl', index=False, header=True)

    # Load the workbook and access the sheet
    wb = load_workbook(output_file_path)
    ws = wb.active

    # Apply formatting to the header row
    header_font = Font(bold=True)
    for cell in ws[1]:
        cell.font = header_font

    # Adjust column widths
    for col in ws.columns:
        max_length = 0
        column = col[0].column_letter  # Get the column name
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = (max_length + 2)
        ws.column_dimensions[column].width = adjusted_width

    # Save the workbook
    wb.save(output_file_path)

    print(f"Results have been saved to {output_file_path}")
else:
    print("Couldn't find relevant data in the sheet")


Results have been saved to B:\Geo Project\Ramanan Sir\T final\vellore climate.xlsx
